In [ ]:
pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 40.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import linear_kernel
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import underthesea
from underthesea import word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
stop_words = open('/content/drive/MyDrive/RS/vietnamese-stopwords.txt')
stop_words = stop_words.read().replace("\n", ",")
stop_words = stop_words.replace(" ", "_")
stop_words = list(stop_words.split(","))
stop_words

In [ ]:
hotel = pd.read_csv('/content/drive/MyDrive/hotels_users_ratings.csv')

In [ ]:
hotel['tags'] = hotel['Location'] + ' ' + hotel['Descriptions'] + ' ' + hotel['Address'] + ' ' + hotel['Name Hotel']
hotel.head()

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating,tags
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1187,Thảo,6,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1284,Tran,5,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."
2,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5866,Tho,6,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."
3,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3033,Tuan,9,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."
4,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1406,Phan,7,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."


In [ ]:
hotel_new = hotel.drop(columns = ['Location', 'Descriptions', 'Address', 'UserID', 'User', 'Rating'])
hotel_new = hotel_new.drop_duplicates()
hotel_new = hotel_new.drop_duplicates(subset='Name Hotel')
hotel_new = hotel_new.reset_index(drop = True)
hotel_new

,URL Hotel,HotelID,Name Hotel,tags
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,4064,Dalat Wind Deluxe Hotel,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."
1,https://www.booking.com/hotel/vn/tropicana-2.v...,1173,Miền Nhiệt Đới 2 Hotel,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Quản..."
2,https://www.booking.com/hotel/vn/reddoorz-near...,2090,Raon Dalat,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách CLB ..."
3,https://www.booking.com/hotel/vn/royal-palace-...,175,Royal Palace,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Công..."
4,https://www.booking.com/hotel/vn/jolie-house-a...,1113,Jolie House,Đà Lạt Tọa lạc ở thành phố Đà Lạt thuộc tỉnh L...
...,...,...,...,...
4501,https://www.booking.com/hotel/vn/hung-long-hos...,1083,Hưng Long Hostel,Huế Tất cả phòng nghỉ tại đây đều có khu vực g...
4502,https://www.booking.com/hotel/vn/le-house-hue....,1194,Le House,Huế Căn hộ này có chỗ đỗ xe riêng miễn phí và...
4503,https://www.booking.com/hotel/vn/pinehill-tu-h...,2196,Pinehill Tu Hieu Hue Homestay,"Huế Các căn tại đây có tầm nhìn ra vườn, khu v..."
4504,https://www.booking.com/hotel/vn/snooze-villa-...,4444,Snooze Garden,Huế Mỗi chỗ nghỉ gắn náy điều hòa tại đây đều ...


In [ ]:
list_loc = pd.DataFrame({'Location'})
list_loc = hotel['Location']
list_loc = list_loc.drop_duplicates()
list_loc = list_loc.reset_index(drop = True)
list_loc

0             Đà Lạt
1             Hà Nội
2    TP. Hồ Chí Minh
3           Vũng Tàu
4            Đà Nẵng
5           Phú Quốc
6             Hội An
7          Nha Trang
8              Sa Pa
9                Huế
Name: Location, dtype: object

In [ ]:
hotel_info = hotel_new.drop(columns = ['tags', 'URL Hotel'])
hotel_info = hotel_info.drop_duplicates()
hotel_info = hotel_info.reset_index(drop = True)
hotel_info

,HotelID,Name Hotel
0,4064,Dalat Wind Deluxe Hotel
1,1173,Miền Nhiệt Đới 2 Hotel
2,2090,Raon Dalat
3,175,Royal Palace
4,1113,Jolie House
...,...,...
4501,1083,Hưng Long Hostel
4502,1194,Le House
4503,2196,Pinehill Tu Hieu Hue Homestay
4504,4444,Snooze Garden


In [ ]:
# viết hàm tiền xử lý dữ liệu
def processing_data(data):
# - chuyển về từ thường
  data = data.lower()
# - Xóa dấu câu, ký tự đặc biệt
  data = re.sub('[^\w\s]', ' ', str(data))
# - tách từ
  data = data.split()
# - xóa stopword
  data = [word for word in data if word not in stop_words]
# - Chuyển từ collection lists of words sang lists of string
  data = ' '.join(data)
  return data

In [ ]:
def tokenize_it(col):
  for i in tqdm(range(len(hotel_new['tags']))):
    try:
      col[i] = word_tokenize(col[i], format = 'text')
    except Exception as ex:
      print(col[i])

In [ ]:
def processing_col(col):
  for i in tqdm(range(len(hotel_new['tags']))):
    try:
      col[i] = processing_data(col[i])
    except Exception as ex:
      print(col[i])

In [ ]:
def csr_to_list(csr_matrix):
  l_list = []
  num = len(csr_matrix.data)
  for i in range(num):
    l_list.append({'index': csr_matrix.indices[i],'values': csr_matrix.data[i]})
  return l_list

In [ ]:
hotel_new['tags'] = hotel_new['tags'].fillna('')

In [ ]:
processing_col(hotel_new['tags'])

  0%|          | 0/4506 [00:00<?, ?it/s]<ipython-input-23-1dc539ca5de6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = processing_data(col[i])
100%|██████████| 4506/4506 [00:31<00:00, 145.30it/s]


In [ ]:
tokenize_it(hotel_new['tags'])

  0%|          | 0/4506 [00:00<?, ?it/s]<ipython-input-22-e69c2982797e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = word_tokenize(col[i], format = 'text')
100%|██████████| 4506/4506 [01:10<00:00, 64.12it/s]


In [ ]:
mapping_1 = pd.Series(hotel_info['Name Hotel'], index = hotel_info.index)
mapping_2 = pd.Series(hotel_info['HotelID'], index = hotel_info.index)
mapping_3 = pd.Series(hotel_info.index, hotel_info['HotelID'])

In [ ]:
# rec theo location
def hotel_by_location(location, num_of_rec):
  location = word_tokenize(location, format = 'text')
  for i in range(len(list_loc)):
    temp = word_tokenize(list_loc[i], format = 'text')
    if(location == temp):
      rec_list = []
      hotel_list = hotel[hotel['Location'] == list_loc[i]]
      mean_rating = hotel_list.groupby('HotelID')['Rating'].mean()
      mean_rating = mean_rating.sort_values(ascending = False)
      list_Id = mean_rating[:num_of_rec]
      list_Id = list_Id.index.tolist()
      for j in range(num_of_rec):
        rec_list.append({list_Id[j]: mapping_1[mapping_3[list_Id[j]]]})
      return rec_list

In [ ]:
hotel_by_location('Huế', 10)


[{63: 'Le Robinet Villa - a journey into Hue citadel soul'},
 {1279: 'Q Garden Villa Homestay'},
 {68: 'Le Home'},
 {262: 'Sunny C Hotel'},
 {4404: "Van's Serene Garden Homestay"},
 {2403: 'Melia Vinpearl Hue'},
 {4256: 'Forest Tag Homestay'},
 {2553: 'Tim Homestay Nera Garden'},
 {1717: 'Ngọc Bích Home Huế'},
 {495: 'K’may Apartment'}]

In [ ]:
keys_list = list(ttt[0].keys())

In [ ]:
ttt[0][list(ttt[0].keys())[0]]

'Le Robinet Villa - a journey into Hue citadel soul'

In [ ]:
df_for_vectorizing = hotel_new['tags']

In [ ]:
# rec theo nội dung tìm kiếm
def hotel_by_search(search, df_for_vectorizing, nums_output):
  list_for_rec = []
  search = search.lower()
  search = re.sub('[^\w\s]', ' ', str(search))
  search = word_tokenize(search, format = 'text')
  df_for_vectorizing_temp = hotel_new['tags'].append(pd.Series([search], index=[len(df_for_vectorizing)]))
  vectorizer = TfidfVectorizer()
  tags_matrix = vectorizer.fit_transform(hotel_new['tags'].append(pd.Series([search], index=[len(df_for_vectorizing)])))
  similarities = linear_kernel(tags_matrix, tags_matrix, dense_output = False)
  ind = len(df_for_vectorizing_temp) - 1
  list_dict = similarities[ind]
  list_dict = csr_to_list(list_dict)
  #sắp xếp list_dict giảm dần theo values, và lấy ra top n
  list_dict = sorted(list_dict, key = lambda x: x['values'], reverse = True)[1:nums_output + 1]
  for j in range(len(list_dict)):
    list_dict[j]['index'] = list_dict[j]['index'].astype(np.int64)
  for i in range(len(list_dict)):
    list_for_rec.append({mapping_2[list_dict[i]['index']]: mapping_1[list_dict[i]['index']]})
  return list_for_rec

In [ ]:
hotel[hotel['HotelID'] == 2057]

In [ ]:
hotel_by_search('Đà Nẵng, thành phố, ban công, hồ, duplex, ăn sáng', df_for_vectorizing, 10)

<ipython-input-129-cf97a158e760>:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_for_vectorizing_temp = hotel_new['tags'].append(pd.Series([search], index=[len(df_for_vectorizing)]))
<ipython-input-129-cf97a158e760>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tags_matrix = vectorizer.fit_transform(hotel_new['tags'].append(pd.Series([search], index=[len(df_for_vectorizing)])))


[{2057: 'SKY VILLAS DUPLEX-INDOOR SWIMMING POOL PARTY'},
 {1825: 'Homestay Penthouse duplex 2 tầng 240m2'},
 {202: 'May - DUPLEX SKY VIEW 01-2'},
 {2911: 'OCEAN VIEW HOME FOURPOINT DA NANG BEACH'},
 {4237: 'Da Lat Ngoi Sao Viet'},
 {4254: 'An Bang Scent Beach Homestay'},
 {1797: 'Đức Anh Bungalow'},
 {4129: 'Aura Apartment Da Lat'},
 {4212: 'Sanayen Villa & Residences'},
 {1930: 'La Casa Hotel'}]

In [ ]:
hotel_new_1 = hotel.drop(columns = ['Descriptions', 'Address', 'UserID', 'User', 'Rating'])
hotel_new_1 = hotel_new_1.drop_duplicates()
hotel_new_1 = hotel_new_1.drop_duplicates(subset='Name Hotel')
hotel_new_1 = hotel_new_1.reset_index(drop = True)
hotel_new_1

In [ ]:
type(hotel_new_1['tags'])

pandas.core.series.Series

In [ ]:
processing_col(hotel_new_1['tags'])
hotel_new_1

  0%|          | 0/4506 [00:00<?, ?it/s]<ipython-input-23-1dc539ca5de6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = processing_data(col[i])
100%|██████████| 4506/4506 [00:32<00:00, 138.57it/s]


,URL Hotel,Location,HotelID,Name Hotel,tags
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,đà lạt tọa lạc thành phố đà lạt hồ xuân hương ...
1,https://www.booking.com/hotel/vn/tropicana-2.v...,Đà Lạt,1173,Miền Nhiệt Đới 2 Hotel,đà lạt tọa lạc thành phố đà lạt quảng trường l...
2,https://www.booking.com/hotel/vn/reddoorz-near...,Đà Lạt,2090,Raon Dalat,đà lạt tọa lạc thành phố đà lạt clb golf dalat...
3,https://www.booking.com/hotel/vn/royal-palace-...,Đà Lạt,175,Royal Palace,đà lạt tọa lạc thành phố đà lạt công viên yers...
4,https://www.booking.com/hotel/vn/jolie-house-a...,Đà Lạt,1113,Jolie House,đà lạt tọa lạc thành phố đà lạt tỉnh lâm đồng ...
...,...,...,...,...,...
4501,https://www.booking.com/hotel/vn/hung-long-hos...,Huế,1083,Hưng Long Hostel,huế tất phòng nghỉ khu vực ghế tv màn hình phẳ...
4502,https://www.booking.com/hotel/vn/le-house-hue....,Huế,1194,Le House,huế hộ chỗ đô xe miễn phí bếp hộ bố trí 3 phòn...
4503,https://www.booking.com/hotel/vn/pinehill-tu-h...,Huế,2196,Pinehill Tu Hieu Hue Homestay,huế tầm vườn khu vực ghế máy giặt bếp tiện ngh...
4504,https://www.booking.com/hotel/vn/snooze-villa-...,Huế,4444,Snooze Garden,huế chỗ nghỉ gắn náy hòa bếp đi kèm tủ lạnh bế...


In [ ]:
tokenize_it(hotel_new_1['tags'])
hotel_new_1

  0%|          | 0/4506 [00:00<?, ?it/s]<ipython-input-22-e69c2982797e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = word_tokenize(col[i], format = 'text')
100%|██████████| 4506/4506 [01:16<00:00, 59.06it/s]


,URL Hotel,Location,HotelID,Name Hotel,tags
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,đà lạt tọa_lạc thành_phố đà lạt hồ xuân hương ...
1,https://www.booking.com/hotel/vn/tropicana-2.v...,Đà Lạt,1173,Miền Nhiệt Đới 2 Hotel,đà lạt tọa_lạc thành_phố đà_lạt quảng_trường l...
2,https://www.booking.com/hotel/vn/reddoorz-near...,Đà Lạt,2090,Raon Dalat,đà lạt tọa_lạc thành_phố đà_lạt clb golf dalat...
3,https://www.booking.com/hotel/vn/royal-palace-...,Đà Lạt,175,Royal Palace,đà lạt tọa_lạc thành_phố đà_lạt công_viên yers...
4,https://www.booking.com/hotel/vn/jolie-house-a...,Đà Lạt,1113,Jolie House,đà lạt tọa_lạc thành_phố đà lạt tỉnh lâm_đồng ...
...,...,...,...,...,...
4501,https://www.booking.com/hotel/vn/hung-long-hos...,Huế,1083,Hưng Long Hostel,huế tất_phòng nghỉ khu_vực ghế tv màn_hình phẳ...
4502,https://www.booking.com/hotel/vn/le-house-hue....,Huế,1194,Le House,huế hộ chỗ đô xe miễn_phí bếp hộ bố_trí 3 phòn...
4503,https://www.booking.com/hotel/vn/pinehill-tu-h...,Huế,2196,Pinehill Tu Hieu Hue Homestay,huế tầm vườn khu_vực ghế máy giặt bếp tiện_ngh...
4504,https://www.booking.com/hotel/vn/snooze-villa-...,Huế,4444,Snooze Garden,huế chỗ nghỉ gắn náy hòa bếp đi kèm tủ_lạnh bế...


In [ ]:
def loc_search_hotel(location, search, num):
  for i in range(len(list_loc)):
    location = word_tokenize(location, format = 'text')
    temp = word_tokenize(list_loc[i], format = 'text')
    if(location == temp):
      list_for_rec = []
      search = search.lower()
      search = re.sub('[^\w\s]', ' ', str(search))
      search = word_tokenize(search, format = 'text')
      df_for_vectorizing = hotel_new_1[hotel_new_1['Location'] == list_loc[i]]
      mapping_4 = pd.Series(df_for_vectorizing['Name Hotel'], df_for_vectorizing.index)
      mapping_5 = pd.Series(df_for_vectorizing['HotelID'], df_for_vectorizing.index)
      df_for_vectorizing = df_for_vectorizing.reset_index(drop = True)
      mapping_6 = pd.Series(hotel_new_1[hotel_new_1['Location'] == list_loc[i]].index, df_for_vectorizing.index)
      df_for_vectorizing_temp = df_for_vectorizing['tags'].append(pd.Series([search + ' ' + location], index=[len(df_for_vectorizing)]))
      vectorizer = TfidfVectorizer()
      tags_matrix = vectorizer.fit_transform(df_for_vectorizing_temp)
      similarities = linear_kernel(tags_matrix, tags_matrix, dense_output = False)
      ind = (len(df_for_vectorizing_temp) - 1)
      list_dict = similarities[ind]
      list_dict = csr_to_list(list_dict)
      #sắp xếp list_dict giảm dần theo values, và lấy ra top n
      list_dict = sorted(list_dict, key = lambda x: x['values'], reverse = True)[1:num + 1]
      for j in range(len(list_dict)):
        list_dict[j]['index'] = list_dict[j]['index'].astype(np.int64)
      for i in range(len(list_dict)):
        list_for_rec.append({mapping_5[mapping_6[list_dict[i]['index']]]: mapping_4[mapping_6[list_dict[i]['index']]]})
      return list_for_rec
  return hotel_by_search(search,hotel_new['tags'], num)

In [ ]:
loc_search_hotel('Sa Pa', 'Thành phố, núi, sông, hồ, biển, mây', 8)

<ipython-input-131-393f8951a02a>:15: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_for_vectorizing_temp = df_for_vectorizing['tags'].append(pd.Series([search + ' ' + location], index=[len(df_for_vectorizing)]))


In [ ]:
type(hotel_new_1[hotel_new_1['Location'] == list_loc[2]]['tags'])

pandas.core.series.Series

In [ ]:
type(list_loc)

pandas.core.series.Series

In [ ]:
test_obj = 'Thành phố, núi, sông, hồ, biển, mây' + ' ' + 'Da Lat'
test_obj

'Thành phố, núi, sông, hồ, biển, mây Da Lat'

In [ ]:
type(list_loc[1])

numpy.int64

In [ ]:
test_s

,URL Hotel,Location,HotelID,Name Hotel,tags


In [ ]:
test_s.append(pd.Series(test_obj))

<ipython-input-88-f526979d5c2d>:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_s.append(pd.Series(test_obj))


0    Thành phố, núi, sông, hồ, biển, mây Da Lat
dtype: object

In [ ]:
import pickle

In [ ]:
pickle.dump(hotel, open('hotel.pkl', 'wb'))
pickle.dump(hotel_new, open('hotel_new.pkl', 'wb'))
pickle.dump(stop_words, open('stop_words.pkl', 'wb'))
pickle.dump(list_loc, open('list_loc.pkl', 'wb'))
pickle.dump(hotel_info, open('hotel_info.pkl', 'wb'))
pickle.dump(mapping_1, open('mapping_1.pkl', 'wb'))
pickle.dump(mapping_2, open('mapping_2.pkl', 'wb'))
pickle.dump(mapping_3, open('mapping_3.pkl', 'wb'))
pickle.dump(df_for_vectorizing, open('df_for_vectorizing.pkl', 'wb'))

In [ ]:
pickle.dump(hotel_new_1, open('hotel_new_1.pkl', 'wb'))

In [ ]:
pickle.dump(hotel, open('hotel.pkl', 'wb'))

In [ ]:
mapping_1 = pickle.load(open('mapping_1.pkl', 'rb'))
mapping_2 = pickle.load(open('mapping_2.pkl', 'rb'))
mapping_3 = pickle.load(open('mapping_3.pkl', 'rb'))
hotel = pickle.load(open('hotel.pkl', 'rb'))
hotel_new = pickle.load(open('hotel_new.pkl', 'rb'))
hotel_new_1 = pickle.load(open('hotel_new_1.pkl', 'rb'))
list_loc = pickle.load(open('list_loc.pkl', 'rb'))
df_for_vectorizing = pickle.load(open('df_for_vectorizing.pkl', 'rb'))
hotel_info = pickle.load(open('mapping_3.pkl', 'rb'))

In [ ]:
hotel[hotel['Name Hotel'] == 'SINH ĐÔI HOUSE']['Descriptions'][4082]

'Các căn tại đây có ban công, TV màn hình phẳng và phòng tắm riêng với vòi xịt/chậu rửa vệ sinh cùng dép đi trong phòng. Các căn còn được trang bị tủ lạnh và ấm đun nước. Khách lưu trú tại nhà nghỉ B&B này có thể thưởng thức bữa sáng buffet. SINH ĐÔI HOUSE nằm cách Hồ Xuân Hương 2,1 km và Công viên Yersin 2,2 km. Sân bay gần nhất là sân bay Liên Khương, nằm trong bán kính 22 km từ chỗ nghỉ.'

In [ ]:
df_for_vectorizing_temp.loc[len(df_for_vectorizing)] = 'Thành phố, núi, sông, hồ, biển, mây Da Lat'

In [ ]:
df_for_vectorizing_temp

0       đà lạt tọa_lạc thành_phố đà lạt hồ xuân hương ...
1       đà lạt tọa_lạc thành_phố đà_lạt quảng_trường l...
2       đà lạt tọa_lạc thành_phố đà_lạt clb golf dalat...
3       đà lạt tọa_lạc thành_phố đà_lạt công_viên yers...
4       đà lạt tọa_lạc thành_phố đà lạt tỉnh lâm_đồng ...
                              ...                        
4502    huế hộ chỗ đô xe miễn_phí bếp hộ bố_trí 3 phòn...
4503    huế tầm vườn khu_vực ghế máy giặt bếp tiện_ngh...
4504    huế chỗ nghỉ gắn náy hòa bếp đi kèm tủ_lạnh bế...
4505    huế hộ tv màn_hình phẳng_hô nằm kính 2 9 tử cấ...
4506           Thành phố, núi, sông, hồ, biển, mây Da Lat
Name: tags, Length: 4507, dtype: object

In [ ]:
list_loc = pickle.load(open('list_loc.pkl', 'rb'))

In [ ]:
hotel_new_1 = pickle.load(open('hotel_new_1.pkl', 'rb'))

In [ ]:
type(pd.DataFrame(df_for_vectorizing))

pandas.core.frame.DataFrame

In [ ]:
hotel.head(1)

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating,tags
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1187,Thảo,6,"Đà Lạt Tọa lạc tại thành phố Đà Lạt, cách Hồ X..."


In [ ]:
list_hotel = hotel.drop(columns = ['Location', 'UserID', 'User', 'Rating', 'tags'])
list_hotel = list_hotel.drop_duplicates()
list_hotel = list_hotel.drop_duplicates(subset='Name Hotel')
list_hotel = list_hotel.reset_index(drop = True)
list_hotel

In [ ]:
list_hotel

,URL Hotel,HotelID,Name Hotel,Descriptions,Address
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê..."
1,https://www.booking.com/hotel/vn/tropicana-2.v...,1173,Miền Nhiệt Đới 2 Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","64 Phan Nhu Thach, ward 1, Đà Lạt, Việt Nam"
2,https://www.booking.com/hotel/vn/reddoorz-near...,2090,Raon Dalat,"Tọa lạc tại thành phố Đà Lạt, cách CLB chơi go...","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam"
3,https://www.booking.com/hotel/vn/royal-palace-...,175,Royal Palace,"Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","Đường Hà Huy Tập 55 49A Đường Hà Huy Tập, Đà L..."
4,https://www.booking.com/hotel/vn/jolie-house-a...,1113,Jolie House,Tọa lạc ở thành phố Đà Lạt thuộc tỉnh Lâm Đồng...,"104 Phù Đổng Thiên Vương, Phường 8, Đà Lạt, Vi..."
...,...,...,...,...,...
4501,https://www.booking.com/hotel/vn/hung-long-hos...,1083,Hưng Long Hostel,Tất cả phòng nghỉ tại đây đều có khu vực ghế n...,"Trần Quang Khải 5 kiệt 6, Huế, Việt Nam"
4502,https://www.booking.com/hotel/vn/le-house-hue....,1194,Le House,Căn hộ này có chỗ đỗ xe riêng miễn phí và bếp...,"27 Kiệt 42 Nguyễn Công Trứ 1, Huế, Việt Nam"
4503,https://www.booking.com/hotel/vn/pinehill-tu-h...,2196,Pinehill Tu Hieu Hue Homestay,"Các căn tại đây có tầm nhìn ra vườn, khu vực g...","72 C Le Ngo Cat Thuy Xuan, Huế, Việt Nam"
4504,https://www.booking.com/hotel/vn/snooze-villa-...,4444,Snooze Garden,Mỗi chỗ nghỉ gắn náy điều hòa tại đây đều có b...,"332/3 Bạch Đằng, Huế, Việt Nam"


In [ ]:
ttt[1][list(ttt[1].values())[1]]

IndexError: ignored

In [ ]:
list_hotel[list_hotel['HotelID'] == 4064][:1]

,URL Hotel,HotelID,Name Hotel,Descriptions
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ..."


In [ ]:
list(ttt[1].keys())
re.sub('[^\w\s]', '', str(list(ttt[1].keys())))

'2496'

In [ ]:
def convert_it(a):
  return int(re.sub('[^\w\s]', '', str(list(a.keys()))))

In [ ]:
list_testtt = []
for i in range(len(ttt)):
  list_testtt.append(convert_it(ttt[i]))

In [ ]:
list_testtt = pd.DataFrame(list_testtt)

In [ ]:
list_testtt

2496

In [ ]:
reccomend_list

In [ ]:
for i in range(len(reccomend_list))

In [ ]:
a =

"['05 Phố Kim Đồng, Thị Trấn Sapa, Lào Cai, Sa Pa, Việt Nam']"

In [ ]:
a.strip('[]\'\\')

'https://www.booking.com/hotel/vn/may-ho-sapa.vi.html?label=gen173nr-1FCAEoggI46AdIKlgEaPQBiAEBmAEquAEXyAEM2AEB6AEB-AECiAIBqAIDuALJm9ybBsACAdICJDE3MzdlYWUwLTU1ZDktNGQzNi1hMGZlLTU5MGRkZDc0ODY5MtgCBeACAQ&sid=842627633388b4367a2fb42d0ca3ab7f&aid=304142&ucfs=1&arphpl=1&dest_id=-3728113&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=23&hapos=48&sr_order=popularity&srpvid=b72a41e405bb0095&srepoch=1669022538&from=searchresults#hotelTmpl'

In [ ]:
test_kkk[1] = list_hotel[list_hotel['HotelID'] == list_testtt[1]]

NameError: ignored

In [ ]:
list_testtt

[2504, 2496, 205, 3232, 4486, 1775, 1168, 4295]

In [ ]:
str(list_hotel[list_hotel['HotelID'] == list_testtt[1]]['Name Hotel'].values).strip('[]\'\\')
str(list_hotel[list_hotel['HotelID'] == list_testtt[1]]['Address'].values).strip('[]\'\\')
str(list_hotel[list_hotel['HotelID'] == list_testtt[1]]['Descriptions'].values).strip('[]\'\\')
str(list_hotel[list_hotel['HotelID'] == list_testtt[1]]['URL Hotel'].values).strip('[]\'\\')

'Mây Hồ Hotel Sapa'

In [ ]:
rec_list = []
for i in range(len(list_testtt)):
    row = []
    for j in range(len(list_testtt)):
        row.append(0)
    rec_list.append(row)

In [ ]:
rec_list = []

In [ ]:
for i in range(len(list_testtt)):
  a = str(list_hotel[list_hotel['HotelID'] == list_testtt[i]]['Name Hotel'].values).strip('[]\'\\')
  b = str(list_hotel[list_hotel['HotelID'] == list_testtt[i]]['Address'].values).strip('[]\'\\')
  c = str(list_hotel[list_hotel['HotelID'] == list_testtt[i]]['Descriptions'].values).strip('[]\'\\')
  d = str(list_hotel[list_hotel['HotelID'] == list_testtt[i]]['URL Hotel'].values).strip('[]\'\\')
  rec_list[i] = (a, b, c, d)

In [ ]:
rec_list[7][3]

'https://www.booking.com/hotel/vn/sapa-panorama.vi.html?label=gen173nr-1FCAEoggI46AdIKlgEaPQBiAEBmAEquAEXyAEM2AEB6AEB-AECiAIBqAIDuALJm9ybBsACAdICJDE3MzdlYWUwLTU1ZDktNGQzNi1hMGZlLTU5MGRkZDc0ODY5MtgCBeACAQ&sid=842627633388b4367a2fb42d0ca3ab7f&aid=304142&ucfs=1&arphpl=1&dest_id=-3728113&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=17&hapos=117&sr_order=popularity&srpvid=3d6a41e98136011a&srepoch=1669022548&from=searchresults#hotelTmpl'

In [ ]:
reccomend_list = [(list_hotel[list_hotel['HotelID'] == hotelid]) for hotelid in list_testtt]

In [ ]:
str(list_hotel[list_hotel['HotelID'] == list_testtt[1]]['Address'].values).strip('[]\'\\')

In [ ]:
len(reccomend_list)

8

In [ ]:
hotel_new['tags'][1]

'đà lạt tọa_lạc thành_phố đà_lạt quảng_trường lâm_viên 1 4 km hồ xuân_hương 1 6 km miền nhiệt_đới 2 hotel cung chỗ nghỉ sảnh wi fi miễn_phí toàn khuôn_viên chỗ đỗ xe miễn_phí sạn 2 dịch_vụ tiền_sảnh phòng hành_lý chỗ nghỉ cung dịch_vụ lễ_tân 24 dịch_vụ phòng tổ_chức tour du_lịch phòng nghỉ_sạn khu vư c ghế tv màn_hình phẳng kênh_truyền_hình vệ_tinh_phòng tắm đi kèm de p đi ng vòi xịt chậu rửa vệ_sinh miền nhiệt_đới 2 hotel cung phòng khu vườn phòng trang ấm đun phòng ga trải giường khăn tắm miền nhiệt_đới 2 hotel nằm công_viên yersin đà lạt 1 8 km clb golf dalat palace 2 8 km sân_bay sân_bay liên_khương nằm sạn 30 km chỗ nghỉ cung dịch_vụ đón sân_bay khoản phụ_phí 64 phan_nhu_thach ward 1 đà lạt viê t nam miền nhiệt_đới 2 hotel'